Spire_Notebook

This notebook reads forecast data from NOAA and observation data from Spire, correlates entries, by date and location, and feeds it to an SKlearn machine learning module, with the goal of improving on the wind velocity forecast.

Read_Grib

Input files
This section reads Grib files from NOAA with weather forecast elements.   The grib files are laid out in multi-dimensional arrrays by latitude, longitude, and elevation.  The grib files are created four times a day, at midnight,  6:00 am, 12:00 noon, and 6:00 pm.  NOAA creates grib files with several forecast lead times, the Spire team is using files created at 6:00 am with a 6 hour lead time. This module uses a start date and end date variable to to determine the date range that it will read in. This module also also uses a latitude and longitude range to limit the forecast data to grid locations in Colorado. 

Output files
A numpy array is created from the Read_Grib section with rows that have latitude, longitude,  wind speeds, temperature, dew point, surface pressure, relative humidity, and the date for each.  The latitude and longitude values in the forecast files serve as the grid for the multi-dimensional array. The Read_Grib notebook reads all grib files for a date range, and aggregates that data.  

In [ ]:
!pip install tabulate    #install tabulate for reporting
!pip install wget

In [ ]:
import Nio
import numpy as np
from numpy import percentile
import datetime
import   pandas as  pd
from pandas import DataFrame
from pandas import concat
from tabulate import tabulate
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from scipy import stats
from operator import itemgetter, attrgetter, methodcaller
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import math
import wget  
from datetime import timedelta, date
import scipy.stats as st
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.metrics import r2_score

In [ ]:
start_year = 2018       # use Nio to read data from the grb2 files from NOAA.
end_year = 2019              
start_day = 1101       # set the start_date and end_date to choose a range of dates to readfrom
end_day = 129
timevalue = '12:00:00'
hhmm = '0600'
fcs = '006'

def daterange(start_date, end_date):      #set up date range for the loop to read forecast data
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
start_date = date(start_year, (int((start_day - (start_day % 100)) / 100)), (start_day % 100))
end_date = date(end_year, (int((end_day - (end_day % 100)) / 100)), (end_day % 100)) + timedelta(days=1)

fdata = []
lat = []
lon = []
date1 = [[[]]]
ele = [1, 2, 3]
la = 0
lat_init = 90.0
while la < 361:
    lat.append(lat_init)
    la += 1
    lat_init -= .5
    
lo = 0
lon_init = 0.0
while lo < 720:
    lon.append(lon_init)
    lo += 1
    lon_init += .5

bottom = 107           #  Set limits for Latitudes
top = 98              #  and Longitudes in 
right = 502            #  in Colorado  to limit the 
left = 518             #  volume of data
fdata=[[ele[0],lat[0],lon[0],99.0,99.0,99.0,99.0,start_date]]
#for date in range(start_date,end_date):
for pdate in daterange(start_date, end_date):
    idate = pdate.strftime("%m%d")
    iyear = pdate.strftime("%Y")
    tt = ""
    
    year = iyear
    
    dt1 = [str(int(year)),'-',str(int((int(idate) - (int(idate) % 100)) / 100)),'-',
        str((int(idate)) % 100),' ',timevalue]
    dt2 = tt.join(dt1)
    datetime2 = dt2
    date_time_str = datetime2
    date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
    t = ""
    a = ['/home/jovyan/spire/Input_Data/gfs_4_',str(year),str(idate),'_',str(hhmm),'_',
    str(fcs),'.grb2']
    file1 = t.join(a) 
    file_name = file1
   
    header_file = Nio.open_file(file_name, "r", format="grib")            #use Nio.open
    fdata3 = header_file.variables['UGRD_P0_L103_GLL0'][0:3,0:,0:]        # read u component of 
    fdata4 = header_file.variables['VGRD_P0_L103_GLL0'][0:3,0:,0:]
    fdata5 = header_file.variables['TMP_P0_L1_GLL0'][0:,0:]
    fdata6 = header_file.variables['DPT_P0_L103_GLL0'][0:,0:]
    fdata7 = header_file.variables['RH_P0_L4_GLL0'][0:,0:]
    fdata8 = header_file.variables['PRMSL_P0_L101_GLL0'][0:,0:]
    fdata9 = header_file.variables['PRES_P0_L1_GLL0'][0:,0:]
    fdata10 = header_file.variables['HGT_P0_L100_GLL0'][0:,0:]
    
    la = top
    el = 0
    
    
    while la < bottom:
        lo = right
         
        while lo < left:
          
            el = 0
            while el < 1:
                
                fdata2=[ele[el],lat[la],lon[lo],fdata3[el,la,lo], fdata4[el,la,lo],
                fdata5[la,lo],fdata6[la,lo],fdata7[la,lo],fdata8[la,lo],
                fdata9[la,lo],
                #fdata10[la,lo],
                date_time_obj]
                 
                fdata.append(fdata2)          #save the forecast data elements to an array
                el +=1
        
            lo += 1
        la += 1
del fdata[0]
len(fdata)

In [ ]:
headers = ["El","Lat","Lon","U WSp","V WSp","Temp","Dew Pt","R Hum","RPMSL","PRES","Date"]
print(tabulate(fdata[0:10],headers=headers))     

Read_Excel

Inputs This section reads in Observation files from Spire, they are observed weather measurements taken at 58 weather stations across Colorado. The observations are taken at various times a day, but all times are not consistent for all weather stations, the best time for consistency is 12:00 noon, so those observations will be used.

Outputs A numpy array is created from the Read_Excel section with rows that have latitude, longitude, wind speeds, temperature, pressure, visibility and cloud ceiling measurements.

In [ ]:
#Set start_day to the first date 
#Set end_day to the day after the last date
#Initialize array for mydata 
#Read the files with observation data.
mydata = [['Stat',0,0,99,1200,22,33,44,55]]

start_year = 2018   # use Pandas to read data from the .xlsx files from Spire
end_year = 2019              
start_day = 1101    # set the start_day, start_year  and end_day, end_year to choose a range of dates to read from
end_day = 129


def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
start_date = date(start_year, (int((start_day - (start_day % 100)) / 100)), (start_day % 100))
end_date = date(end_year, (int((end_day - (end_day % 100)) / 100)), (end_day % 100)) + timedelta(days=1)

for pdate in daterange(start_date,end_date):
    idate = pdate.strftime("%m%d")
    iyear = pdate.strftime("%Y")
    year = iyear
    
    
    t = ""
    a = ['/home/jovyan/spire/Input_Data/obs',str(year),str(idate),'.xlsx']
    file1 = t.join(a) 
    file = file1
    # 
    xl = pd.ExcelFile(file) # Print the sheet names
    #print(xl.sheet_names) # Load a sheet into a DataFrame by name: df1
    df1 = xl.parse('Sheet1')
    df = pd.read_excel(file, sheet_name='Sheet1')


    #print("Column headings:")
    StationID = df['Station ID']
    Latitude = df['Latitude']
    Longitude = df['Longitude']
    Elevation = df['Elevation']
    Time = df['Time']
    Temperature = df['Temperature']
    U = df['U']
    V = df['V']
    WindSpeed = df['Wind Speed']
    Visibility = df['Visibility']
    Ceiling = df['Ceiling']
    #print(df.columns)
    #for i in range(10):
    #   print '%-12i%-12i' % (10 ** i, 20 ** i)
    #print("Station ID   Latitude  Longitude  Elevation Time                 Temperature   U   V   Wind Speed ")

    i = 0
    #while loop to build array
    while i < len(df):
        mydata2=[StationID[i],Latitude[i],Longitude[i],Elevation[i],Time[i],Temperature[i],U[i],V[i],WindSpeed[i]]
        mydata.append(mydata2)
        i+=1     
del mydata[0]

Grid_Corr

The Grid_Corr section reads in numpy arrays from Read_Grib and Read_Excel and correlates grid points from the forecast data to weather station grid points. It is set up to correlate those points using 3 different methods, one assigns the station grid point to the closest forecast grid point. The second method calculates the four forecast grid points that surround each station grid point, and the third similar, but assigns wind speed values using an average of the 4 surrounding points. Later in the process, we intend to evaluate these three methods to determine which is optimal.



In [ ]:
headers = ["St ID","Lat","Lon","Elev", "Time","Temp","U","V","Wind Speed"]
print(tabulate(mydata[0:10],headers=headers))


In [ ]:
x2 = np.array(mydata)
x5 = np.array(x2[0:,1:10])
x5a = np.array(x2[0:,1:3])

In [ ]:
headers = ["Lat","Lon","Elev", "Time","Temp","U","V","Wind Speed"]
print(tabulate(x5[0:10],headers=headers))

In [ ]:
#create a file with lat, lon, grid lat, grid lon, and all observed variables
j = 0
i = 0
m = len(x5)
n = 10
x6 = np.array(x5[0:,0:])
#
for i in range(len(x5)):
    x6[i,0] = x5[i,0]
    x6[i,1] = x5[i,1]
    x6[i,4] = x5[i,2]
    #x6[i,5] = x5[i,3]
    #x6[i,5] = 00.0
    x6[i,6] = x5[i,4]
    x6[i,7] = x5[i,5]
    #x6[i,8] = x5[i,6]
    #x6[i,9] = x5[i,7]
    a=x6[i,0]
    c = a - (a % 1)
   
    if x6[i,0] - c < .25:         #  calculate the closest grid latitude and longitudes to each station location
        x6[i,2] = c
    elif x6[i,0] - c < .5:
        x6[i,2] = a + (.5 -(a % 1))
    elif x6[i,0] - c < .75:
        x6[i,2] = a + (.5 -(a % 1))
    else:
        x6[i,2] = a + (1 - (a % 1))
    
    b=x6[i,1]
    d = b - (b % 1)
  
    if x6[i,1] - d < .25:
        x6[i,3] = d
    elif x6[i,1] - d < .5:
        x6[i,3] = b + (.5 -(b % 1))
    elif x6[i,1] - d < .75:
        x6[i,3] = b + (.5 -(b % 1))
    else:
        x6[i,3] = b + (1 - (b %1))

In [ ]:
i=0                                            # this step inserts the time stamp into the array 
x7 = [[0.0,0.0,0.0,0.0,99,1200,22,33,44,55]]   # initialize x7 with dummy data
while i < len(x6):
    xa=[x6[i,0],x6[i,1],x6[i,2],x6[i,3],x6[i,4],x5[i,3],x6[i,6],x6[i,7],x5[i,6],x5[i,7]]
    x7.append(xa)
    i+=1     
del x7[0]
x7a = np.array(x7)

In [ ]:
x8 = [[0.0,0.0,0.0,0.0,99,1200,22,33,44,55]]   # initialize x8 with dummy data
x8a = [0.0,0.0,0.0,0.0,99,1200,22,33,44,55]    # initialize x8a with dummy data
j=0
i=0
for i in range(len(x7)):                       # remove records with filler data
    if x7a[i,9] == 1e+11:
        ii = 2
    elif x7a[i,5].hour != 12:                  # remove records with times other than 12:00
        ii = 2
    else:
        x8a = x7[i]
        x8.append(x8a)
        j +=1
del x8[0]
x8b = np.array(x8)
x8c = np.array(x8b[:,0:4])

In [ ]:
headers=["Lat","Lon","Grid_Lat","Grid_Lon","Elev","Time","Temp","U","V","Wind Sp"]
print(tabulate(x8b[0:10],headers=headers))

In [ ]:
#create a file with lat, lon, 4 surroundong Grid lat points, and all observed variables
j = 0
i = 0
m = len(x5)
n = 10
x10 = np.array(x5[0:,0:])
#x6 = [[0 for x in range(m)] for y in range(n)]

for i in range(len(x5)):
    x10[i,0] = x5[i,0]
    x10[i,1] = x5[i,1]
    #
    
    #
    a=x10[i,0]            # Calculate the grid latitudes above and below each station latitude
    c = a - (a % 1)
   
    if x10[i,0] - c < .5:   # find the latitude above and below
        x10[i,2] = c
        x10[i,4] = c + .5
    else:
        x10[i,2] = c + .5
        x10[i,4] = c + 1
 
    
    b=x10[i,1]          # Calculate the grid longitudes to the right and left of each station longitude
    d = b - (b % 1)
  
    if x10[i,1] - d < .5:    # find the longitude to the right and left
        x10[i,3] = d
        x10[i,5] = d + .5
    else:
        x10[i,3] = d + .5
        x10[i,5] = d + 1
        
    if x10[i,0] - x10[i,2] < .25:   # find the closest grid lat
        x10[i,6] = x10[i,2]
        
    elif x10[i,0] - x10[i,2] < .5:
        x10[i,6] = x10[i,2] + .5
        
    elif x10[i,0] - x10[i,2] < .75:
        x10[i,6] = x10[i,2]
        
    else:
        x10[i,6] = x10[i,2] + 5
        
    if x10[i,1] - x10[i,3] < .25:  # find the closest grid lon
        x10[i,7] = x10[i,3]
        
    elif x10[i,1] - x10[i,3] < .5:
        x10[i,7] = x10[i,3] + .5
        
    elif x10[i,1] - x10[i,3] < .75:
        x10[i,7] = x10[i,3] 
        
    else:
        x10[i,7] = x10[i,5] + .5

In [ ]:
i=0
x11 = [[0.0,0.0,0.0,0.0,0.0, 0.0,0.0,0.0,99,1200,22,33,44,55]]  #initialize x11 with dummy variables
while i < len(x6):
    xa=[x10[i,0],x10[i,1],x10[i,2],x10[i,3],x10[i,4],x10[i,5],x10[i,6],x10[i,7],x5[i,2],x5[i,3],x5[i,4],x5[i,5],x5[i,6],x5[i,7]]
    x11.append(xa)
    i+=1     
del x11[0]
x11a = np.array(x11)
len(x11a)

In [ ]:
headers=["Lat","Lon","Gla1","Glo1","Gla2","Glo2","CLa1","CLo1","Elev","Time","Temp","U","V","Wind Sp"]
print(tabulate(x11[0:10],headers=headers))

In [ ]:
# this step does data cleanup
# it removes records with invalid wind speed and 
# removes records with timestamps not = 12:00

x12 = [[0.0,0.0,0.0,0.0,0.0,0.0,0.0,99,1200,22,33,44,55]]   # initialize x12 with dummy data
x12a = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,99,1200,22,33,44,55]    # initialize x12a with dummy data
j=0
i=0
for i in range(len(x11)):                                   # take out records with filler data
    if x11a[i,13] == 1e+11:
        ii = 2
    elif x11a[i,9].hour != 12:                              #  take out records with timestamps not equal to 12:00
        ii = 2
    else:
        x12a = x11a[i]
        x12.append(x12a)
        j +=1
del x12[0]
len(x12)

In [ ]:
x13 = sorted(x12, key=itemgetter(0,1))  #Sort by lat and lon

In [ ]:
headers=["Lat","Lon","Gla1","Glo1","Gla2","Glo2","CLa1","CLo1","Elev","Time","Temp","U","V","Wind Sp"]
print(tabulate(x13[0:10],headers=headers))

In [ ]:
x13a = np.array(x13)
x14 = [[0.0,0.0,0.0,0.0,0.0,0.0,0.0,99,1200,22,33,44,55]]     # initialize x14 with dummy data
x14a = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,99,1200,22,33,44,55]      # initialize x14a with dummy data
j= len(x13) - 1
i=0
for i in range(j):
    if (x13a[i,0] == x13a[i+1,0] and x13a[i,1] == x13a[i+1,1]):   # remove records with duplicate lat and lon
        i += 1
    else:
        x14a = x13[i]
        x14.append(x14a)
        i +=1
del x14[0]
x15a = np.array(x14)
x15 = np.array(x15a[:,:8])
len(x15)

In [ ]:
x15g = sorted(x15, key=itemgetter(6,7))  #Sort by  closest lat lon
x15h = np.array(x15g)
len(x15h)

In [ ]:
x15c = [[0.0,0.0,0.0,0.0,0.0,0.0,0.0,99,12]]         # initialize x15c with dummy data
x15d = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,99,12]           # initialize x15d with dummy data
j= len(x15h) - 1
i=0
for i in range(j):
    if (x15h[i,6] == x15h[i+1,6] and x15h[i,7] == x15h[i+1,7]):  # remove records with duplicate lat and lon
        i += 1
    else:
        x15d = x15h[i]
        x15c.append(x15d)
        i +=1
del x15c[0]
x15e = np.array(x15c)
x15f = np.array(x15e)
len(x15f)

In [ ]:
headers=["Lat","Lon","Gla1","Glo1","Gla2","Glo2","CLa1","CLo1","Elev","Time","Temp","U","V","Wind Sp"]
print(tabulate(x15f[0:10],headers=headers))   #one record for each Station Location,  sorted by lat and lon

Extract_Data

The Extract_Data section matches data values from the forecast files to the observation files and prepares files to be input to the Sklearn module. The Sklearn module requires that the forecast and observation files are identical in size and that entries are sorted so that they correlate to eachother. The Extract_Data notebook uses the merge function from Pandas to match up records from each file by grid location and date.

In [ ]:
headers = ["El","Lat","Lon","U WSp","V WSp","Temp","Dew Pt","R Hum","RPMSL","PRES","Date"]
print(tabulate(fdata[0:10],headers=headers)) 

In [ ]:
fdata2 = sorted(fdata, key=itemgetter(10,1,2))  #Sort by date, lat, and then lo
fdata2a = np.array(fdata2)
len(fdata2)

In [ ]:
fdata3 = np.insert(fdata2a,5,   #   convert the u and v components of wind speed to 
                                #   wind speed
                   
((((fdata2a[:,3])**2) + ((fdata2a[:,4])**2))**.5),axis=1)
fdata3a=np.array(fdata3)

In [ ]:
headers = ["El","Lat","Lon","U WSp","V WSp","Wind SP","Temp","Dew Pt","RH","RPMSL","PRES","Date"]
print(tabulate(fdata3a[0:10],headers=headers))     

In [ ]:
headers = ["Lat","Lon","GLat1","GLon1","GLat2","GLon2","CLat","CLon","Elev","Time",
"Temp", "U", "V", "WindSp" ]
print(tabulate(x13[0:10],headers=headers))  

The following cell calculates the forecast wind speed to correlate to each weather station observed wind  speed. This step creates an array that will be used for subsequent analysis steps. The layout of the array is listed below.  The code calculates the forecast wind speed 3 different ways:

  1     Wind speed from the forecast grid point closest to the weather station location.
  
  2     Wind speed averaged from the 4 forecast grid points that surround the weather station location.
  
  3     Wind speed from a weighted average from the 4 forecast grid points that surround the weather 
      station location.  
      
  **** Warning  ******  this cell is long running

In [ ]:
def wtaveragecalc2(b):  #bilinear interpolation function to calculate a weighted average for 4 surrounding grid points
    return ( (1/(np.abs((cc - aa) * (bb - dd)))) *                          # 1 / (x1 - x2)(y1 - y2)
    (x13aa[i,b - 6] * ((np.abs((x13aa[i,0] - cc) * (x13aa[i,1] - dd)))) +   # f(x11)*(x2 - x)(y2 - y)
    x13aa[i,b - 5] * ((np.abs((x13aa[i,0] - cc) * (x13aa[i,1] - bb)))) +    # f(x21)*(x - x1)(y2 - y)
    x13aa[i,b - 4] * ((np.abs((x13aa[i,0] - aa) * (x13aa[i,1] - dd)))) +    # f(x11)*(x2 - x)(y - y1)
    x13aa[i,b - 3] * ((np.abs((x13aa[i,0] - aa) * (x13aa[i,1] - bb))))))    # f(x22)*(x - x1)(y - y1)
#Bilinear Interpolation formula from 
# https://en.wikipedia.org/wiki/Bilinear_interpolation
def averagecalc(b):
    return (x13aa[i,b - 5] + x13aa[i,b - 4] + x13aa[i,b - 3] + x13aa[i, b - 2]) / 4

In [ ]:
   # add columns to x13aa for 4 surrounding grid 
   # point windspeed values and the closest 
   # grid point wind speed value
    
x13ee = np.array(x13)
x13aa = np.append(x13ee,np.zeros([len(x13),28]),1)




i = 0                                                                   
for i in range(len(x13aa)):                                               
     aa = x13aa[i,4] # upper lat                                        
     bb = x13aa[i,3] # left lon
     cc = x13aa[i,2] # lower lat
     dd = x13aa[i,5] # right lon
     ee = x13aa[i,6] # closest lat
     ff = x13aa[i,7] # closest lon
    
     dt = x13aa[i,9] # date time stamp
    
     #  match the observed windspeeds to the four forecast grid points above, 
     #  below, and to the right and left
    
     temp1a = np.where((fdata3a[:,1] == aa ) & (fdata3a[:,2] == bb) & (fdata3a[:,11] == dt),fdata3a[:,5],-1)
     itemindex1 = np.where(temp1a != -1)
     itemindex1a = np.array(itemindex1)
     aaa = int(itemindex1a[0,0])
        
     x13aa[i,14] = fdata3a[aaa,5]   # add wind speed for the upper left grid point
     x13aa[i,21] = fdata3a[aaa,6]   # add temperature for the upper left grid point
     x13aa[i,28] = fdata3a[aaa,9]   # add PRMSL for the upper left grid point
     x13aa[i,35] = fdata3a[aaa,10]  # add Pressure for the upper left grid point
    
     temp1b = np.where((fdata3a[:,1] == aa ) & (fdata3a[:,2] == dd) & (fdata3a[:,11] == dt),fdata3a[:,5],-1)
     itemindex2 = np.where(temp1b != -1)
     itemindex2a = np.array(itemindex2)
     bbb = int(itemindex2a[0,0])
     x13aa[i,15] = fdata3a[bbb,5]   # add wind speed for the upper right grid point
     x13aa[i,22] = fdata3a[bbb,6]   # add temperature for the upper right grid point
     x13aa[i,29] = fdata3a[bbb,9]   # add PRMSL for the upper right grid point
     x13aa[i,36] = fdata3a[bbb,10]  # add Pressure for the upper right grid point
     
     temp1c = np.where((fdata3a[:,1] == cc ) & (fdata3a[:,2] == bb) & (fdata3a[:,11] == dt),fdata3a[:,5],-1)
     itemindex3 = np.where(temp1c != -1)
     itemindex3a = np.array(itemindex3)
     ccc = int(itemindex3a[0,0])
     x13aa[i,16] = fdata3a[ccc,5]   # add wind speed for the lower left grid point
     x13aa[i,23] = fdata3a[ccc,6]   # add temperature for the lower left grid point
     x13aa[i,30] = fdata3a[ccc,9]   # add PRMSL for the lower left grid point
     x13aa[i,37] = fdata3a[ccc,10]  # add Pressure for the lower left grid point
     
     temp1d = np.where((fdata3a[:,1] == cc ) & (fdata3a[:,2] == dd) & (fdata3a[:,11] == dt),fdata3a[:,5],-1)
     itemindex4 = np.where(temp1d != -1)
     itemindex4a = np.array(itemindex4)
     ddd = int(itemindex4a[0,0])
     x13aa[i,17] = fdata3a[ddd,5]   # add wind speed for the lower right grid point
     x13aa[i,24] = fdata3a[ddd,6]   # add temperature for the lower right grid point
     x13aa[i,31] = fdata3a[ddd,9]   # add PRMSL for the lower right grid point
     x13aa[i,38] = fdata3a[ddd,10]  # add Pressure for the lower right grid point
     
     temp1e = np.where((fdata3a[:,1] == ee ) & (fdata3a[:,2] == ff) & (fdata3a[:,11] == dt),fdata3a[:,5],-1)
     itemindex5 = np.where(temp1e != -1)
     itemindex5a = np.array(itemindex5)
     eee = int(itemindex5a[0,0])
     x13aa[i,18] = fdata3a[eee,5]   # add wind speed for the closest grid point
     x13aa[i,25] = fdata3a[eee,6]   # add temperature for the closest grid point
     x13aa[i,32] = fdata3a[eee,9]   # add PRMSL for the closest grid point
     x13aa[i,39] = fdata3a[eee,10]   # add Pressure for the closest grid point
    
     # Calculate the average forecasted windspeed of the four grid points surrounding each 
     # observation. 
    
     x13aa[i,19] = averagecalc(19) # Wind SP
     x13aa[i,26] = averagecalc(26) # Temp
     x13aa[i,33] = averagecalc(33) # PRMSL
     x13aa[i,40] = averagecalc(40) # Press
    
     # Calculate a weighted  average forecasted windspeed of the four grid points 
     # surrounding each observation.
    
     x13aa[i,20] = wtaveragecalc2(20)   # Wind SP
    
     x13aa[i,27] = wtaveragecalc2(27)   # Temp
    
     x13aa[i,34] = wtaveragecalc2(34)   # PRMSL
    
     x13aa[i,41] = wtaveragecalc2(41)   # Press

X13aa file layout

  Field and Description
  
•  0  Station Latitude

•  1  Station Longitude

•  2  Grid Latitude below the Station Latitude

•  3  Grid Longitude to the left of the  Station Longitude

•  4  Grid Latitude above the Station Longitude

•  5  Grid Longitude to the right of the Station Longitude

•  6  Closest Grid Latitude to the Station Latitude

•  7  Closest Grid Longitude to the Station Longitude

•  8  Elevation of the station

•  9  Date//Time of the observation

• 10 Temperature 

• 11 Observed U component of windspeed in meters per second

• 12 Observed V component of windspeed in meters per second

• 13 Observed windspeed for the station location in meters per second

• 14 Forecasted wind speed for the upper left grid point

• 15 Forecasted wind speed for the upper right grid point

• 16 Forecasted wind speed for the lower left grid point

• 17 Forecasted wind speed for the lower right grid point

• 18 Forecasted wind speed for the closest grid point

• 19 Average forecasted wind speed for the 4 surrounding grid points

• 20 Weighted average forecasted wind speed for the 4 surrounding grid points

• 21 Forecasted temperature for the upper left grid point

• 22 Forecasted temperature for the upper right grid point

• 23 Forecasted temperature for the lower left grid point

• 24 Forecasted temperature for the lower right grid point

• 25 Forecasted temperature for the closest grid point

• 26 Average forecasted temperature for the 4 surrounding grid points

• 27 Weighted average forecasted temperature for the 4 surrounding grid points

• 28 Forecasted Pressure reduced to Mean Sea Level for the upper left grid point

• 29 Forecasted Pressure reduced to Mean Sea Level for the upper right grid point

• 30 Forecasted Pressure reduced to Mean Sea Level for the lower left grid point

• 31 Forecasted Pressure reduced to Mean Sea Level for the lower right grid point

• 32 Forecasted Pressure reduced to Mean Sea Level for the closest grid point

• 33 Average forecasted Pressure reduced to Mean Sea Level for the 4 surrounding grid
     points
   
• 34 Weighted average forecasted Pressure reduced to Mean Sea Level for the 4 surrounding 
     grid points
• 35 Forecasted Pressure for the upper left grid point

• 36 Forecasted Pressure for the upper right grid point

• 37 Forecasted Pressure for the lower left grid point

• 38 Forecasted Pressure for the lower right grid point

• 39 Forecasted Pressure for the closest grid point

• 40 Average forecasted Pressure for the 4 surrounding grid points

• 41 Weighted average forecasted Pressure for the 4 surrounding grid points

In [ ]:
headers = ["Lat","Lon","GLat1","GLon1","GLat2","GLon2","CLat","CLon","Elev","Time","Temp",
           "U", "V", "WindSp","UL WS","UR WS","LL WS", "LR WS","Cl WS","AVG WS","Wtd Avg"]
print(tabulate(x13aa[0:2],headers=headers))  

In [ ]:
x16 = sorted(x15f, key=itemgetter(6,7))  #Sort by  closest lat lon
x16a = np.array(x16)
len(x16a)

In [ ]:
headers = ["Lat","Lon","GLat1","GLon1","GLat2","GLon2","CLat","CLon"]
print(tabulate(x16a[0:10],headers=headers))   

In [ ]:
forext13ab = np.array(x13aa[0:,[19,26,33,40]])  #
obsext13ab = np.array(x13aa[0:,[13]])  #


In [ ]:
forext13ac = np.array(x13aa[0:,[20,27,34,41]])  #four points
obsext13ac = np.array(x13aa[0:,[13]])  #weighted average

In [ ]:
#Read the file with possible grid points
file2 = "/home/jovyan/spire/Input_Data/Grid_Locations.xlsx"

xl2 = pd.ExcelFile(file2) # Print the sheet names
#print(xl.sheet_names) # Load a sheet into a DataFrame by name: df1
df12 = xl2.parse('Sheet1')
df2 = pd.read_excel(file2, sheet_name='Sheet1')

In [ ]:
Latitude = df2['Latitude']
Longitude = df2['Longitude']
headers2 = ["Latitude","Longitude"]
mydata3 = [[Latitude[0],Longitude[0]]]
i = 1
#while loop to build array
while i < len(df2):
    mydata4=[Latitude[i],Longitude[i]]
    mydata3.append(mydata4)
    i+=1     

In [ ]:
x2 = np.array(mydata)
x4 = np.array(mydata3)
x5 = np.array(x2[0:,1:10])
x5a = np.array(x2[0:,1:3])

In [ ]:
plt.figure(figsize=(15,10))
def scatterplots(x) :
    p = x.shape[1]
    q = 'Forecast Grid Points'
    for r in range(p - 1) :          # row variables loop:
        for c in range(r + 1, p) :   # column variables loop:
            plt.subplot(p - 1, p - 1, (p - 1)*r + c)
            plt.scatter(x[:, c], x[:, r], label=str(q), color = 'b', s = 40)
scatterplots(x4)
def scatterplots(x) :
    p = x.shape[1]
    q = 'Observation Locations'
    for r in range(p - 1) :          # row variables loop:
        for c in range(r + 1, p) :   # column variables loop:
            plt.subplot(p - 1, p - 1, (p - 1)*r + c)
            plt.scatter(x[:, c], x[:, r], label=str(q), color = 'r', s = 40)
scatterplots(x5a)
plt.title("Colorado Forecast Grid Points and Weather Stations" ,fontsize=16)
plt.xlabel("Longitude", fontsize=12)
plt.ylabel("Latitude", fontsize=12)
plt.legend(loc='lower right', shadow=True)

plt.show()

In [ ]:
#Plot weather station locations over closest Grid Points
plt.figure(figsize=(12,8))
x7 = np.array(x6[0:,2:4])
def scatterplots(x) :
    p = x.shape[1]
    q = 'Forecast Grid Points'
    for r in range(p - 1) :          # row variables loop:
        for c in range(r + 1, p) :   # column variables loop:
            plt.subplot(p - 1, p - 1, (p - 1)*r + c)
            plt.scatter(x[:, c], x[:, r], label=str(q), color = 'b', s = 10)
scatterplots(x4)
def scatterplots(x) :
    p = x.shape[1]
    q = 'Station Locations mapped to Grid Points'
    for r in range(p - 1) :          # row variables loop:
        for c in range(r + 1, p) :   # column variables loop:
            plt.subplot(p - 1, p - 1, (p - 1)*r + c)
            plt.scatter(x[:, c], x[:, r], label=str(q),  color = 'r', s = 40)
# suptitle('Weather Station Locations', fontsize=16)
scatterplots(x7)
plt.title("Colorado Weather Station Locations mapped over Closest Forecast Grid Points", fontsize=16)
plt.xlabel("Longitude", fontsize=12)
plt.ylabel("Latitude", fontsize=12)
    
plt.legend(loc='lower right', shadow=True)
plt.show()

In [ ]:
#Compare Observed and Forecast Wind Speeds
WindData = np.array(x13aa)
OWS=WindData[:,13]
FWSC=WindData[:,18]
FWSAV=WindData[:,19]
FWSWTAV=WindData[:,20]


In [ ]:
plt.figure(figsize=(12,8))
box_plot_data=[OWS,FWSC,FWSAV,FWSWTAV]
plt.boxplot(box_plot_data,patch_artist=True,
labels=['Observed Wind Speed','Forecast Windspeed\nClosest Grid Point',
    'Forecast Windspeed\nAvg of 4 surrounding \nGrid Points',
    'Forecast Windspeed\nWeighted Avg of Four \nSurrounding Grid Points'])
plt.title("Observed Wind Speeds compared with Forecast Wind Speeds", fontsize=16)
plt.ylabel("Wind Speeds in Meters per Second", fontsize=12)
plt.text(2, 25, r'The gold line in the boxplot is the median')
plt.text(2, 24, r'The bottom of the box is the First Quartile')
plt.text(2, 23, r'The top of the box is the Third Quartile')
plt.text(2, 22, r'The length of the box is the Inter Quartile Range (IQR)')
plt.text(2, 21, r'The whiskers are (1.5 * IQR) above the Third Quartile and below the First Quartile')
plt.text(2, 20, r'The observations above the whisker are outliers')
plt.show()

In [ ]:
FWSCA = np.zeros(len(FWSC))
FWSAVA = np.zeros(len(FWSC))
FWSWTAVA = np.zeros(len(FWSC))
Diff1 = np.zeros(len(FWSC))

for i in range(len(FWSC)):
    FWSCA[i] = np.around(FWSC[i],decimals = 2)
    FWSAVA[i] = np.around(FWSAV[i],decimals = 2)
    FWSWTAVA[i] = np.around(FWSWTAV[i],decimals = 2)

In [ ]:
for i in range(len(FWSC)):
    Diff1[i] = np.around(OWS[i] - FWSCA[i],decimals=1)
Diff2=Diff1[0:]
plt.figure(figsize=(12,8))
plt.hist(Diff2,bins=60)
plt.title("Observed - Forecast From Closest Points     Difference Histogram")
plt.ylabel("Frequency")
plt.ylim(bottom=0)
plt.ylim(top=1400)
plt.text(10, 1000, r'Difference of Observed and Forecaset Winspeed')
plt.text(10, 950, r'Windspeed in Meters per Second')

In [ ]:
for i in range(len(FWSC)):
    Diff1[i] = np.around(OWS[i] - FWSAVA[i],decimals=1)
Diff2=Diff1[0:]
plt.figure(figsize=(12,8))
plt.hist(Diff2,bins=60)
plt.title("Observed - Forecast Average from Surrounding Points Difference Histogram")
plt.ylabel("Frequency")
plt.ylim(bottom=0)
plt.ylim(top=1400)
plt.text(10, 1000, r'Difference of Observed and Forecaset Winspeed')
plt.text(10, 950, r'Windspeed in Meters per Second')

In [ ]:
for i in range(len(FWSC)):
    Diff1[i] = np.around(OWS[i] - FWSWTAVA[i],decimals=1)
Diff2=Diff1[0:]
plt.figure(figsize=(12,8))
plt.hist(Diff2,bins=60)
plt.title("Observed - Forecast Weighted Average from Surrounding Points Difference Histogram")
plt.ylabel("Frequency")
plt.ylim(bottom=0)
plt.ylim(top=1400)
plt.text(10, 1000, r'Difference of Observed and Forecaset Winspeed')
plt.text(10, 950, r'Windspeed in Meters per Second')

In [ ]:
OWSbyDate = np.array(WindData[0:,[13,9]])
FWSCbyDate = np.array(WindData[0:,[18,9]])
FWSAVbyDate = np.array(WindData[0:,[19,9]])
FWSWTAVbyDate = np.array(WindData[0:,[20,9]])

In [ ]:
WindData2=np.array(WindData[:,[9,13,18,8]])  # Set up a line graph with observed and forecast windsppeds for comparison
for station in range(len(x15f)):
    OWSStatbyDate = []                         # Choose one station and graph both variables over time
    #OWSStatbyDate = [1.1, 2018-11-1]  # initialize with dummy data
    for ii in range(len(WindData)):
        if WindData[ii,0] == x15f[station,0] and WindData[ii,1] == x15f[station,1]:
            OWSStatbyDate.append(WindData2[ii,0:4])
    OWSStatbyDate2 = np.array(OWSStatbyDate)
    titlestring1 = "Observed and Forecast Windspeed For One Station by Date \n  Station Location  Latitude = "
    titlestring2 = str(np.around(x15f[station,0],decimals=2)) + "   Longitude = " + str(np.around(x15f[station,1],decimals=2))
    titlestring3 = " \n Station Elevation = " + str(OWSStatbyDate[0][3]) + " meters"
    titlestring = titlestring1 + titlestring2 + titlestring3
    plt.figure(figsize=(12,8))

    plt.title(titlestring) 
    plt.xlabel("Date", fontsize = 12)
    plt.ylabel("Wind Speed in meters per second", fontsize = 12)
    #plt.text(1,WindData2[14,0], r'Difference of Observed and Forecaset Winspeed')
    plt.xlim([datetime.date(2018, 11, 1), datetime.date(2019, 1, 29)])
    plt.xticks(rotation='vertical')
    plt.plot(OWSStatbyDate2[:,0],OWSStatbyDate2[:,1],color='b',label='Observed Wind Speed')
    plt.plot(OWSStatbyDate2[:,0],OWSStatbyDate2[:,2],color='r',label='Forecast Wind Speed')
    plt.legend(loc='upper right', shadow=True)
    plt.show()

In [ ]:
WindData2=np.array(WindData[:,[9,13,18,8,0,1]])  # Set up a line graph with observed and forecast windsppeds for comparison

OWSStatbyDate1 = []
OWSStatbyDate2 = []   
OWSStatbyDate3 = []  
OWSStatbyDate4 = [] 

suspects = [[37.45,254.13],[37.95,252.1],[38.1,253.83],[38.53,253.07],
            [38.78,251.94],[38.81,253.88],[39.65,253.08],[40.04,253.63],[40.15,254.84],[40.52,253.13]]

for ii in range(len(WindData2)):
        
        test = np.isin(suspects, WindData2[ii,[4,5]])
        for iii in range(len(suspects)):
            type(test[iii])
            if test[iii][0] == True  and test[iii][1] == True:
                OWSStatbyDate4.append(WindData2[ii,0:4])
                
        if WindData2[ii,3] <= 2000:
            OWSStatbyDate1.append(WindData2[ii,0:4])
        elif WindData2[ii,3] <= 3000:
            OWSStatbyDate2.append(WindData2[ii,0:4])
        else:
            OWSStatbyDate3.append(WindData2[ii,0:4])
OWSStatbyDate11 = np.array(OWSStatbyDate1)
OWSStatbyDate22 = np.array(OWSStatbyDate2)
OWSStatbyDate33 = np.array(OWSStatbyDate3)
OWSStatbyDate44 = np.array(OWSStatbyDate4)
titlestring1 = "Observed and Forecast Windspeed For All Stations  \n Color Coded by Elevation  "
 
titlestring = titlestring1
plt.figure(figsize=(12,8))
plt.title(titlestring) 
plt.ylabel("Observed Wind Speed in meters per second", fontsize = 12)
plt.xlabel("Forecast Wind Speed in meters per second", fontsize = 12)
 
plt.xticks(rotation='vertical')
plt.scatter(OWSStatbyDate33[:,2],OWSStatbyDate33[:,1],color='y',s=5,label='Stations above 3000 meters')

plt.scatter(OWSStatbyDate22[:,2],OWSStatbyDate22[:,1],color='b',s=5,label='Stations below 3000 meters')
plt.scatter(OWSStatbyDate11[:,2],OWSStatbyDate11[:,1],color='r',s=5,label='Stations below  2000 meters')
plt.scatter(OWSStatbyDate44[:,2],OWSStatbyDate44[:,1],color='g',s=5,label='Suspect Stations')
plt.plot(OWSStatbyDate11[:,1],OWSStatbyDate11[:,1],color='r',label='X = Y')
plt.legend(loc='upper right', shadow=True)
plt.show()

Analysis

Analysis-Linear Regression The Analysis section summarize wind speed for observation data,weather forecast data and merged dataset, and compare which wind speed(closet or averaged)we should put into the model. The Analysis notebook use df.dataframe to generate different linear regression model and use backward selection and transformation to make R^2 as big as possible.

In [ ]:
#Weather Forecast wind speed 
#fdata = np.array(forecast_for_merged)
data=x13aa[:,18]
quartiles = percentile(data, [25, 50, 75])
data_min, data_max = data.min(), data.max()
print('Min: %.3f' % data_min)
print('Q1: %.3f' % quartiles[0])
print('Median: %.3f' % quartiles[1])
print('Q3: %.3f' % quartiles[2])
print('Max: %.3f' % data_max)

In [ ]:
# wind speed for observation

odata1 = np.array(x13aa[:,13])
data1=odata1
quartiles = percentile(data1, [25, 50, 75])
data1_min, data1_max = data1.min(), data1.max()
print('Min: %.3f' % data1_min)
print('Q1: %.3f' % quartiles[0])
print('Median: %.3f' % quartiles[1])
print('Q3: %.3f' % quartiles[2])
print('Max: %.3f' % data1_max)

In [ ]:
plt.figure(figsize=(12,8))
box_plot_data=[data,data1]
plt.boxplot(box_plot_data,patch_artist=True,labels=['Weather Forecast\nClosest Grid Point','Observation Wind Speed'])
#The upper and lower line represent Q1-1.5*IQR and Q3+1.5*IQR
#IQR=Q3-Q1 which is interquatile range
plt.title("Observed Wind Speeds compared with Forecast Wind Speeds", fontsize=16)
plt.ylabel("Wind Speeds in Meters per Second", fontsize=12)
plt.text(1, 25, r'The gold line in the boxplot is the median')
plt.text(1, 24, r'The bottom of the box is the First Quartile')
plt.text(1, 23, r'The top of the box is the Third Quartile')
plt.text(1, 22, r'The length of the box is the Inter Quartile Range (IQR)')
plt.text(1, 21, r'The whiskers are (1.5 * IQR) above the Third Quartile and below the First Quartile')
plt.text(1, 20, r'The observations above the whisker are outliers')
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.kdeplot(data)
sns.kdeplot(data1)
plt.title("Observed Wind Speeds compared with Forecast Wind Speeds", fontsize=16)
plt.xlabel("Wind Speeds in Meters per Second", fontsize=12)
plt.text(10,0.30,r'The blue line represents Weather Forecast at Closest Grid Point')
plt.text(10,0.25,r'The Orange line represents Observation Wind Speed')

In [ ]:
plt.figure(figsize=(10, 10), dpi=80)
plt.scatter(data, data1, label = "Relationship",s=5,color='r')
plt.xlabel('Weather Forecast - axis')
plt.ylabel('Observation - axis')
plt.title('Comparison')
plt.legend()
plt.plot(data,data, linestyle='solid')

In [ ]:
merged_data = pd.DataFrame({'Glat':x13aa[:,6],'Glon':x13aa[:,7],'Elevation':x13aa[:,8],'WSobs':x13aa[:,13],'Date':x13aa[:,9],
        'WSu':x13aa[:,11],'WSv':x13aa[:,12],'WSfor':x13aa[:,18],'Temp':x13aa[:,25],'PRMSL':x13aa[:,32],
        'PRES':x13aa[:,39]})
df=DataFrame(merged_data,columns=["Latitude","Longitude","Elevation","WSobs","Date","WSu","WSv","WSfor","Temp","PRMSL","PRES"])
X1=df[['WSfor']]
Y1=df['WSobs']
regr = linear_model.LinearRegression()
regr.fit(X1, Y1)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
X2 = sm.add_constant(X1)
model = sm.OLS(Y1.astype(float), X1.astype(float)).fit()
predictions = model.predict(X1) 
model.summary()

In [ ]:
X1=df[['Elevation','WSfor','Temp','PRMSL','PRES']]
Y1=df['WSobs']
regr = linear_model.LinearRegression()
regr.fit(X1, Y1)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
X2 = sm.add_constant(X1)
model = sm.OLS(Y1.astype(float), X1.astype(float)).fit()
predictions = model.predict(X1) 
model.summary()

In [ ]:
#Backward Selection
X1=df[['Elevation','WSfor','Temp','PRES']]
Y1=df['WSobs']
regr = linear_model.LinearRegression()
regr.fit(X1, Y1)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
X2 = sm.add_constant(X1)
model = sm.OLS(Y1.astype(float), X1.astype(float)).fit()
predictions = model.predict(X1) 
model.summary()

In [ ]:
df.index[df['Elevation']>3000].tolist()

In [ ]:
df=df.drop(df.index[720:1007])
X1=df[['Elevation','WSfor','Temp','PRES']]
Y1=df['WSobs']
regr = linear_model.LinearRegression()
regr.fit(X1, Y1)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
X2 = sm.add_constant(X1)
model = sm.OLS(Y1.astype(float), X1.astype(float)).fit()
predictions = model.predict(X1) 
model.summary()

In [ ]:
df=df.drop(df.index[2465:2660])
X1=df[['Elevation','WSfor','Temp','PRES']]
Y1=df['WSobs']
regr = linear_model.LinearRegression()
regr.fit(X1, Y1)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
X2 = sm.add_constant(X1)
model = sm.OLS(Y1.astype(float), X1.astype(float)).fit()
predictions = model.predict(X1) 
model.summary()

In [ ]:
df=df.drop(df.index[4799:5072])
X1=df[['Elevation','WSfor','Temp','PRES']]
Y1=df['WSobs']
regr = linear_model.LinearRegression()
regr.fit(X1, Y1)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
X2 = sm.add_constant(X1)
model = sm.OLS(Y1.astype(float), X1.astype(float)).fit()
predictions = model.predict(X1) 
model.summary()

In [ ]:
df=df.drop(df.index[5306:5415])
X1=df[['Elevation','WSfor','Temp','PRES']]
Y1=df['WSobs']
regr = linear_model.LinearRegression()
regr.fit(X1, Y1)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
X2 = sm.add_constant(X1)
model = sm.OLS(Y1.astype(float), X1.astype(float)).fit()
predictions = model.predict(X1) 
model.summary()

In [ ]:
df.index[df['Elevation']>3000].tolist()

In [ ]:
df=df.drop(df.index[5906:6059])
X1=df[['Elevation','WSfor','Temp','PRES']]
Y1=df['WSobs']
regr = linear_model.LinearRegression()
regr.fit(X1, Y1)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
X2 = sm.add_constant(X1)
model = sm.OLS(Y1.astype(float), X1.astype(float)).fit()
predictions = model.predict(X1) 
model.summary()

In [ ]:
df=df.drop(df.index[6169:6359])
X1=df[['Elevation','WSfor','Temp','PRES']]
Y1=df['WSobs']
regr = linear_model.LinearRegression()
regr.fit(X1, Y1)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
X2 = sm.add_constant(X1)
model = sm.OLS(Y1.astype(float), X1.astype(float)).fit()
predictions = model.predict(X1) 
model.summary()

In [ ]:
df=df.drop(df.index[6802:7065])
X1=df[['Elevation','WSfor','Temp','PRES']]
Y1=df['WSobs']
regr = linear_model.LinearRegression()
regr.fit(X1, Y1)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
X2 = sm.add_constant(X1)
model = sm.OLS(Y1.astype(float), X1.astype(float)).fit()
predictions = model.predict(X1) 
model.summary()

In [ ]:
plt.figure(figsize=(12, 8), dpi=80)
plt.scatter(df['WSobs'], df['WSfor'], label = "Accuracy",s=5)
plt.xlabel('Weather forecast-Elevation < 3000')
plt.ylabel('Observation-')
plt.title('Comparison')
plt.legend() 
plt.plot(Y1,Y1+0 , linestyle='solid')

In [ ]:
plt.figure(figsize=(12, 8), dpi=80)
plt.scatter(Y1, model.fittedvalues, label = "Accuracy",s=5)
plt.xlabel('Observation - axis')
plt.ylabel('Fitted Wind Speed- axis')
plt.title('Comparison')
plt.legend() 
plt.plot(Y1,Y1+0 , linestyle='solid')

In [ ]:
df = df.assign(squareroot_WSobs=df['WSobs']**(1/2))
X1=df[['WSfor','Temp','PRES']]
Y1=df['squareroot_WSobs']
regr = linear_model.LinearRegression()
regr.fit(X1, Y1)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
X2 = sm.add_constant(X1)
model = sm.OLS(Y1.astype(float), X1.astype(float)).fit()
predictions = model.predict(X1) 
model.summary()

In [ ]:
plt.figure(figsize=(10, 10), dpi=80)
plt.scatter(Y1, model.fittedvalues, label = "Accuracy",s=5)
plt.xlabel('Observation - axis')
plt.ylabel('Fitted Wind Speed- axis')
plt.title('Comparison')
plt.legend() 
plt.plot(Y1,Y1+0 , linestyle='solid')

In [ ]:
print(np.corrcoef(Y1,model.fittedvalues ))

Neural Network Steps: using sklearn model
The Neural Netwok inputs Forecast data and Wind Speeds from observations.
We want to make a prediction model and use cross validation to test the data. The test data set is 0.2 of the whole data. Inside the trainning data, the dataset is devided into trainning data and 0.1 validation data, which is used in early stopping to avoid overfit. 
Here is the 3 month data whose data size is 11510. 
The four variables of the data are:
19 Average forecasted wind speed for the 4 surrounding grid points, 
26 Average forecasted temperature for the 4 surrounding grid points, 
33 Average forecasted Pressure reduced to Mean Sea Level for the 4 surrounding grid points, 
40 Average forecasted Pressure for the 4 surrounding grid points
Estimated running time: 15s

In [ ]:
avscore=0
for t in range(5):
    forext13ab = np.array(x13aa[0:,[19,26,33,40]])  #
    obsext13ab = np.array(x13aa[0:,[13]])  #
    X= forext13ab.astype(float)
    Y= obsext13ab.astype(float)
    Y.shape=(len(x13aa),)
    X.shape=(len(x13aa),4)
    X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2)
    scaler = StandardScaler()
    scaler.fit(X_train)
    StandardScaler(copy=True, with_mean=True, with_std=True)
    # Now apply the transformations to the data:
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    mlp = MLPRegressor(hidden_layer_sizes=(5,3),verbose=True,early_stopping=True,max_iter=1500)
    mlp.fit(X_train,y_train)
    avscore+=mlp.score(X_test, y_test)
s=avscore/5
print("Average test set score: %f" % s)

Then we make prediction with 1 month data. The data size is only 3706. First we inport the data.Estimated running time: 9s

In [ ]:
avscore=0
with open('/home/jovyan/spire/Input_Data/Test_Extract_Data_obsext13ab_pickle', 'rb') as f:
    obsext13ab = pickle.load(f)
with open('/home/jovyan/spire/Input_Data/Test_Extract_Data_forext13ab_pickle', 'rb') as f:
    forext13ab = pickle.load(f)
for t in range(10):
    X= forext13ab.astype(float)
    Y= obsext13ab.astype(float)
    Y.shape=(3706,)
    X.shape=(3706,4)
    X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2)
    scaler = StandardScaler()
    scaler.fit(X_train)
    StandardScaler(copy=True, with_mean=True, with_std=True)
    # Now apply the transformations to the data:
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    mlp = MLPRegressor(hidden_layer_sizes=(5,3),verbose=True,early_stopping=True,max_iter=1500)
    mlp.fit(X_train,y_train)
    avscore+=mlp.score(X_test, y_test)
s=avscore/10
print("Average test set score: %f" % s)

As we can see here. The larger data size helps improve the prediction of the model. The test score is improved from 0.187 to 0.234. we will continue explore the 3 month dataset.we test the similiar transformation as the linear regression model. square root y. And we only use one pressure data to predict. Estimated running time: 23s

In [ ]:
avscore=0
for t in range(10):
    forext13ab = np.array(x13aa[0:,[19,26,40]])  #
    obsext13ab = np.array(x13aa[0:,[13]])  #
    X= forext13ab.astype(float)
    Y= np.sqrt(obsext13ab.astype(float))
    Y.shape=(len(x13aa),)
    X.shape=(len(x13aa),3)
    X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2)
    scaler = StandardScaler()
    scaler.fit(X_train)
    StandardScaler(copy=True, with_mean=True, with_std=True)
    # Now apply the transformations to the data:
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    mlp = MLPRegressor(hidden_layer_sizes=(5,3),verbose=True,early_stopping=True,max_iter=1500)
    mlp.fit(X_train,y_train)
    avscore+=mlp.score(X_test, y_test)
s=avscore/10
print("Average test set score: %f" % s)

We can see here our prediction score is not improved. Then we take the elevation of the station into consideration.
Estimated running time 25s

In [ ]:
avscore=0
for t in range(10):
    forext13ab = np.array(x13aa[0:,[8,19,26,40]])  #
    obsext13ab = np.array(x13aa[0:,[13]])  #
    X= forext13ab.astype(float)
    Y= obsext13ab.astype(float)
    Y.shape=(len(x13aa),)
    X.shape=(len(x13aa),4)
    X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2)
    scaler = StandardScaler()
    scaler.fit(X_train)
    StandardScaler(copy=True, with_mean=True, with_std=True)
    # Now apply the transformations to the data:
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    mlp = MLPRegressor(hidden_layer_sizes=(5,3),verbose=True,early_stopping=True,max_iter=1500)
    mlp.fit(X_train,y_train)
    avscore+=mlp.score(X_test, y_test)
s=avscore/10
print("Average test set score: %f" % s)

Surprisingly, the test score improved a lot.This time we will try to split wind into two direction.
Estimated running time: 25s

In [ ]:
avscore=0
for t in range(10):
    forext13ab = np.array(x13aa[0:,[8,11,12,26,40]])  #
    obsext13ab = np.array(x13aa[0:,[13]])  #
    X= forext13ab.astype(float)
    Y= obsext13ab.astype(float)
    Y.shape=(len(x13aa),)
    X.shape=(len(x13aa),5)
    X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2)
    scaler = StandardScaler()
    scaler.fit(X_train)
    StandardScaler(copy=True, with_mean=True, with_std=True)
    # Now apply the transformations to the data:
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    mlp = MLPRegressor(hidden_layer_sizes=(5,3),verbose=True,early_stopping=True,max_iter=1500)
    mlp.fit(X_train,y_train)
    avscore+=mlp.score(X_test, y_test)
s=avscore/10
print("Average test set score: %f" % s)

The test score decreased. Adding the real temperature data into consideration.Estimated running time: 23s

In [ ]:
avscore=0
for t in range(10):
    forext13ab = np.array(x13aa[0:,[8,19,26,40,10]])  #
    obsext13ab = np.array(x13aa[0:,[13]])  #
    X= forext13ab.astype(float)
    Y= obsext13ab.astype(float)
    Y.shape=(len(x13aa),)
    X.shape=(len(x13aa),5)
    X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2)
    scaler = StandardScaler()
    scaler.fit(X_train)
    StandardScaler(copy=True, with_mean=True, with_std=True)
    # Now apply the transformations to the data:
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    mlp = MLPRegressor(hidden_layer_sizes=(5,3),verbose=True,early_stopping=True,max_iter=1500)
    mlp.fit(X_train,y_train)
    avscore+=mlp.score(X_test, y_test)
s=avscore/10
print("Average test set score: %f" % s)

The testing score is improving again. However we can not observe the real data of the temperation before it happend.It is also possible that this improvement is caused by more predictors.
Next we would add lontitude and lontitude into our model.

In [ ]:
avscore=0
for t in range(10):
    forext13ab = np.array(x13aa[0:,[6,7,19,26,40,10]])  #
    obsext13ab = np.array(x13aa[0:,[13]])  #
    X= forext13ab.astype(float)
    Y= obsext13ab.astype(float)
    Y.shape=(len(x13aa),)
    X.shape=(len(x13aa),6)
    X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2)
    scaler = StandardScaler()
    scaler.fit(X_train)
    StandardScaler(copy=True, with_mean=True, with_std=True)
    # Now apply the transformations to the data:
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    mlp = MLPRegressor(hidden_layer_sizes=(5,3),verbose=True,early_stopping=True,max_iter=1500)
    mlp.fit(X_train,y_train)
    avscore+=mlp.score(X_test, y_test)
s=avscore/10
print("Average test set score: %f" % s)

The data is not improved by the lontitude and latitude

For now the best neural network model is the one using:
8 Elevation of the station
19 Average forecasted wind speed for the 4 surrounding grid points
26 Average forecasted temperature for the 4 surrounding grid points
40 Average forecasted Pressure for the 4 surrounding grid points
10 Temperature
as predictor.
The structure of the Neural Network is also adjusted. By changing structure to 5-10-4-1, the prediction score is improved 0.1-0,2.

In [ ]:
avscore=0
for t in range(10):
    forext13ab = np.array(x13aa[0:,[8,19,26,40,10]])  #
    obsext13ab = np.array(x13aa[0:,[13]])  #
    X= forext13ab.astype(float)
    Y= obsext13ab.astype(float)
    Y.shape=(len(x13aa),)
    X.shape=(len(x13aa),5)
    X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2)
    scaler = StandardScaler()
    scaler.fit(X_train)
    StandardScaler(copy=True, with_mean=True, with_std=True)
    # Now apply the transformations to the data:
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    mlp = MLPRegressor(hidden_layer_sizes=(5,3),verbose=True,early_stopping=True,max_iter=1500)
    mlp.fit(X_train,y_train)
    avscore+=mlp.score(X_test, y_test)
s=avscore/10
print("Average test set score: %f" % s)

In [ ]:
resid = mlp.predict(X_test)-y_test
sse = sum(resid**2)
k = 86
n = 10500
AIC= 2*k - 2*np.log(sse)
BIC = n*np.log(sse/n) + k*np.log(n)

In [ ]:
print("AIC is: %f" % AIC)
print("BIC is: %f" % BIC)

In [ ]:
np.corrcoef(y_test, mlp.predict(X_test))

Here are the correlation and r-square of the forecast wind speed and observation wind speed.

In [ ]:
X=np.array(x13aa[0:,[19]])
X=np.round(X.astype(float),decimals=5)
Y.shape=(len(x13aa),)
X.shape=(len(x13aa),)
print(np.corrcoef(X, Y))
print(r2_score(Y,X))

As we can see, the prediction of neural network improved the performance of original prediction.